# Actividad 1 – Machine Learning II (Churn)

**Objetivo:** predecir la probabilidad de fuga de clientes (churn) en una empresa de telecomunicaciones mediante el uso de Regresión Logística.

Para este propósito, se comparan distintos enfoques de modelamiento:

- Un modelo base, construido a partir de variables previamente preprocesadas.

- Un modelo con transformaciones polinomiales, de grado 2, aplicadas sobre un conjunto seleccionado de variables numéricas.

- Modelos regularizados, que incorporan penalizaciones L2 y L1 (Elastic Net) para controlar la complejidad del modelo.

- La evaluación del desempeño se realiza mediante validación cruzada estratificada (k-fold), considerando métricas y herramientas de análisis acordes a un problema de clasificación con posible desbalance de clases, tales como:

- Matriz de confusión y métricas asociadas (accuracy, precision, recall y F1).

- Curva ROC y su respectiva métrica AUC-ROC.

- Curva Precision–Recall y PR-AUC (Average Precision), especialmente relevante en contextos donde la clase de interés es minoritaria.

In [ ]:
# =========================
# 0) Librerías
# =========================
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_curve,
    auc,
    precision_recall_curve,
    average_precision_score
)

import warnings
warnings.filterwarnings("ignore")

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)


##Primer paso

Se utiliza el dataset data-churn.csv, el cual contiene información de clientes de una empresa de telecomunicaciones. A partir de este conjunto de datos, se definen las siguientes variables:

Variable objetivo: Churn, de tipo binaria, donde Yes indica que el cliente abandona el servicio (1) y No que permanece (0).

Variables numéricas: MonthlyCharges, TotalCharges y tenure, las cuales representan características cuantitativas del comportamiento y antigüedad del cliente.

Variables categóricas: Contract, InternetService, PaymentMethod y PhoneService, que describen distintos atributos del tipo de servicio y condiciones contractuales.

In [ ]:
# =========================
# 1) Carga de datos
# =========================
df = pd.read_csv("data-churn.csv")
df.head()


In [ ]:
# Variable objetivo: Yes/No -> 1/0
df["Churn"] = df["Churn"].map({"Yes": 1, "No": 0})

print("Tamaño del dataset:", df.shape)
print("\nProporción de clases (Churn):")
display(df["Churn"].value_counts(normalize=True).rename("proporción"))


In [ ]:
# Estadísticos básicos (rápidos)
display(df.describe(include="all").transpose().head(25))


## Segundo paso

En esta etapa se definen las principales decisiones de preprocesamiento, orientadas a preparar los datos de forma adecuada para el entrenamiento de los modelos:

Tratamiento de valores faltantes: se aplica una imputación simple, utilizando la mediana para las variables numéricas y la categoría más frecuente para las variables categóricas, con el fin de evitar la pérdida de información.

Variables categóricas: se utiliza One-Hot Encoding con el parámetro drop='first', lo que permite representar las categorías en forma numérica y, al mismo tiempo, evitar problemas de multicolinealidad perfecta.

Variables numéricas: se realiza una estandarización mediante StandardScaler, asegurando que todas las variables se encuentren en una escala comparable y facilitando la convergencia del modelo de Regresión Logística.

In [ ]:
# =========================
# 2) Selección de variables
# =========================
numeric_features = ["MonthlyCharges", "TotalCharges", "tenure"]
categorical_features = ["Contract", "InternetService", "PaymentMethod", "PhoneService"]

X = df[numeric_features + categorical_features].copy()
y = df["Churn"].copy()

# Asegurar tipo numérico en columnas numéricas (por ejemplo TotalCharges viene como texto)
for col in numeric_features:
    X[col] = pd.to_numeric(X[col], errors="coerce")

print("Nulos por columna en X:")
display(X.isnull().sum())


In [ ]:
# =========================
# 3) Preprocesador base
# =========================
numeric_transformer_base = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(drop="first", handle_unknown="ignore"))
])

preprocessor_base = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer_base, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)


## Función de evaluación con k-fold estratificado

Se define una función de evaluación que permite analizar el desempeño de los modelos de forma consistente a través de los distintos folds de la validación cruzada estratificada. Esta función:

Calcula las principales métricas de clasificación en cada fold, incluyendo accuracy, precision, recall, F1, ROC-AUC y PR-AUC.

Resume los resultados mediante una tabla que presenta el promedio y la desviación estándar de cada métrica, facilitando la comparación entre modelos.

Genera las curvas ROC promedio y Precision–Recall promedio, obtenidas a partir del promedio de las curvas de cada fold mediante interpolación, lo que permite una evaluación visual más estable del desempeño global del modelo.

In [ ]:
def evaluate_model_cv(model, X, y, cv_splits=5, title="Modelo"):
    skf = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=RANDOM_STATE)

    fold_rows = []
    roc_curves = []
    pr_curves = []

    # Grillas comunes para promediar curvas
    mean_fpr = np.linspace(0, 1, 200)
    mean_recall = np.linspace(0, 1, 200)

    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y), start=1):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:, 1]

        # Métricas
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, zero_division=0)
        rec = recall_score(y_test, y_pred, zero_division=0)
        f1 = f1_score(y_test, y_pred, zero_division=0)

        fpr, tpr, _ = roc_curve(y_test, y_proba)
        roc_auc = auc(fpr, tpr)

        p, r, _ = precision_recall_curve(y_test, y_proba)
        pr_auc = average_precision_score(y_test, y_proba)

        fold_rows.append({
            "fold": fold,
            "accuracy": acc,
            "precision": prec,
            "recall": rec,
            "f1": f1,
            "roc_auc": roc_auc,
            "pr_auc": pr_auc
        })

        # Curvas interpoladas para promediar
        tpr_interp = np.interp(mean_fpr, fpr, tpr)
        tpr_interp[0] = 0.0
        roc_curves.append(tpr_interp)

        # PR: interpolar precision en función de recall (recall viene descendente a veces)
        # Ordenamos por recall ascendente
        order = np.argsort(r)
        r_sorted = r[order]
        p_sorted = p[order]
        p_interp = np.interp(mean_recall, r_sorted, p_sorted)
        pr_curves.append(p_interp)

    results = pd.DataFrame(fold_rows)

    # Resumen
    summary = pd.DataFrame({
        "mean": results.drop(columns=["fold"]).mean(),
        "std": results.drop(columns=["fold"]).std()
    })

    print(f"\n=== {title} ===")
    display(results)
    display(summary)

    # Matriz de confusión global (juntando predicciones de folds de forma reproducible)
    # (Para mantener simple: re-ejecutamos y acumulamos)
    y_true_all, y_pred_all, y_proba_all = [], [], []
    for train_idx, test_idx in skf.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:, 1]
        y_true_all.extend(y_test)
        y_pred_all.extend(y_pred)
        y_proba_all.extend(y_proba)

    cm = confusion_matrix(y_true_all, y_pred_all)
    print("Matriz de confusión global (todos los folds):")
    print(cm)
    print("\nReporte de clasificación global:")
    print(classification_report(y_true_all, y_pred_all, digits=3))

    # Curvas promedio
    mean_tpr = np.mean(roc_curves, axis=0)
    mean_tpr[-1] = 1.0
    mean_roc_auc = auc(mean_fpr, mean_tpr)

    mean_precision = np.mean(pr_curves, axis=0)
    mean_pr_auc = auc(mean_recall, mean_precision)  # área aproximada sobre curva promedio

    plt.figure()
    plt.plot(mean_fpr, mean_tpr, label=f"ROC promedio (AUC≈{mean_roc_auc:.3f})")
    plt.plot([0, 1], [0, 1], linestyle="--")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate (Recall)")
    plt.title(f"Curva ROC promedio – {title}")
    plt.legend()
    plt.show()

    plt.figure()
    plt.plot(mean_recall, mean_precision, label=f"PR promedio (AUC≈{mean_pr_auc:.3f})")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title(f"Curva Precision–Recall promedio – {title}")
    plt.legend()
    plt.show()

    return {"per_fold": results, "summary": summary, "cm": cm}


## Tercer paso

Se entrena un modelo de Regresión Logística utilizando las variables previamente preprocesadas. Con el objetivo de aproximar un modelo sin regularización significativa, se emplea una penalización L2 con un valor de C suficientemente grande, lo que en la práctica equivale a una regularización muy débil y permite analizar el comportamiento del modelo base sin un control estricto de complejidad.


In [ ]:
# =========================
# 4) Modelo base
# =========================
log_reg_base = Pipeline(steps=[
    ("preprocess", preprocessor_base),
    ("clf", LogisticRegression(
        penalty="l2",
        C=1e6,           # C grande ~ regularización muy débil (casi 'none')
        solver="lbfgs",
        max_iter=2000,
        random_state=RANDOM_STATE
    ))
])

results_base = evaluate_model_cv(log_reg_base, X, y, cv_splits=5, title="LogReg Base (sin polinomios)")


## Cuarto paso

De acuerdo con la pauta de la actividad, se incorporan transformaciones polinomiales de grado 2, las cuales incluyen términos de interacción, con el objetivo de capturar relaciones más complejas entre las variables numéricas.

Estas transformaciones se aplican sobre un subconjunto de variables consideradas relevantes. En este caso, y manteniendo la coherencia con el código base utilizado, se seleccionan las siguientes variables numéricas:

- MonthlyCharges

- TotalCharges

- tenure


In [ ]:
# =========================
# 5) Preprocesador con polinomios (grado 2)
# =========================
selected_num = ["MonthlyCharges", "TotalCharges", "tenure"]

numeric_transformer_poly = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("poly", PolynomialFeatures(degree=2, include_bias=False))
])

preprocessor_poly = ColumnTransformer(
    transformers=[
        ("poly", numeric_transformer_poly, selected_num),
        ("cat", categorical_transformer, categorical_features),
    ]
)


In [ ]:
# Modelo con polinomios (sin regularización fuerte)
log_reg_poly = Pipeline(steps=[
    ("preprocess", preprocessor_poly),
    ("clf", LogisticRegression(
        penalty="l2",
        C=1e6,
        solver="lbfgs",
        max_iter=3000,
        random_state=RANDOM_STATE
    ))
])

results_poly = evaluate_model_cv(log_reg_poly, X, y, cv_splits=5, title="LogReg + Polinomios (grado 2)")


## Paso 4. Regularización (penalizaciones)

Se evalúan al menos dos variantes de regularización con el fin de controlar la complejidad del modelo:

L2 (Ridge): penaliza la magnitud de los coeficientes, lo que generalmente produce modelos más estables y reduce el riesgo de sobreajuste.

L1 (Lasso) o Elastic Net: introduce esparsidad en los coeficientes, permitiendo que algunas variables tengan una contribución nula y facilitando, de este modo, la selección automática de variables.

Para la selección de los hiperparámetros se utiliza como métrica principal PR-AUC (Average Precision), ya que resulta especialmente adecuada en problemas de clasificación con desbalance de clases, como el caso del churn.

In [ ]:
# =========================
# 6) GridSearch – L2 (Ridge)
# =========================
param_grid_l2 = {
    "clf__penalty": ["l2"],
    "clf__solver": ["lbfgs"],
    "clf__C": [0.01, 0.1, 1.0, 10.0, 100.0]
}

base_pipeline_poly = Pipeline(steps=[
    ("preprocess", preprocessor_poly),
    ("clf", LogisticRegression(max_iter=4000, random_state=RANDOM_STATE))
])

grid_l2 = GridSearchCV(
    base_pipeline_poly,
    param_grid_l2,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE),
    scoring="average_precision",
    n_jobs=-1
)

grid_l2.fit(X, y)
print("Mejores hiperparámetros (L2):", grid_l2.best_params_)
best_model_l2 = grid_l2.best_estimator_


In [ ]:
results_l2 = evaluate_model_cv(best_model_l2, X, y, cv_splits=5, title="LogReg + Polinomios + L2 (tuneado)")


In [ ]:
# =========================
# 7) GridSearch – Elastic Net (incluye L1 como caso límite)
# =========================
param_grid_en = {
    "clf__penalty": ["elasticnet"],
    "clf__solver": ["saga"],
    "clf__l1_ratio": [0.2, 0.5, 0.8],
    "clf__C": [0.01, 0.1, 1.0, 10.0]
}

pipeline_en = Pipeline(steps=[
    ("preprocess", preprocessor_poly),
    ("clf", LogisticRegression(max_iter=6000, random_state=RANDOM_STATE))
])

grid_en = GridSearchCV(
    pipeline_en,
    param_grid_en,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE),
    scoring="average_precision",
    n_jobs=-1
)

grid_en.fit(X, y)
print("Mejores hiperparámetros (Elastic Net):", grid_en.best_params_)
best_model_en = grid_en.best_estimator_


In [ ]:
results_en = evaluate_model_cv(best_model_en, X, y, cv_splits=5, title="LogReg + Polinomios + Elastic Net (tuneado)")


## Comparación final de modelos

Se construye una tabla resumen que presenta, para cada alternativa evaluada, las métricas de desempeño expresadas como promedio y desviación estándar, lo que permite comparar de manera clara y consistente el rendimiento de los distintos modelos.


In [ ]:
def extract_summary(res_dict, model_name):
    s = res_dict["summary"]["mean"].copy()
    s_std = res_dict["summary"]["std"].copy()
    out = pd.DataFrame({
        "metric_mean": s,
        "metric_std": s_std
    })
    out["model"] = model_name
    return out.reset_index().rename(columns={"index": "metric"})

summary_all = pd.concat([
    extract_summary(results_base, "Base"),
    extract_summary(results_poly, "Polinomios g2"),
    extract_summary(results_l2, "Polinomios g2 + L2"),
    extract_summary(results_en, "Polinomios g2 + ElasticNet"),
], ignore_index=True)

pivot = summary_all.pivot(index="metric", columns="model", values="metric_mean")
pivot_std = summary_all.pivot(index="metric", columns="model", values="metric_std")

print("Promedios por métrica:")
display(pivot)

print("Desviación estándar por métrica:")
display(pivot_std)


**¿Qué tan desbalanceado está el churn y por qué la accuracy puede ser engañosa?**

En el problema de churn, la cantidad de clientes que efectivamente abandonan el servicio suele ser menor que la de quienes permanecen. Esto genera un desbalance de clases. En este contexto, un modelo que predice mayoritariamente que los clientes no se irán (churn = 0) puede alcanzar una accuracy alta, aun cuando falle en identificar a muchos de los clientes que sí se van.
Por esta razón, la accuracy por sí sola no es suficiente. Es fundamental analizar la matriz de confusión y métricas como recall y F1 para la clase churn, así como PR-AUC, que es más informativa cuando las clases están desbalanceadas.

**¿Qué modelo seleccionar y por qué?**

En problemas de churn, el modelo más adecuado no es necesariamente el que maximiza la accuracy, sino aquel que logra un buen equilibrio entre identificar correctamente a los clientes que se irán y mantener un nivel razonable de errores.
En este trabajo se comparan tres enfoques: un modelo base, uno con transformaciones polinomiales y modelos regularizados. La selección final se basa principalmente en métricas como PR-AUC y F1, además de la estabilidad de los resultados a lo largo de los distintos folds de la validación cruzada.

**Efecto de las transformaciones polinomiales en el rendimiento y la complejidad**

Las transformaciones polinomiales permiten capturar relaciones más complejas entre las variables, lo que puede mejorar el desempeño del modelo. Sin embargo, también aumentan el número de variables y la complejidad del modelo, lo que incrementa el riesgo de sobreajuste si no se controla adecuadamente.

**Efecto de la regularización (L2 y Elastic Net)**

La regularización ayuda a controlar la complejidad del modelo. La penalización L2 tiende a reducir la magnitud de los coeficientes, lo que suele traducirse en modelos más estables y con mejor capacidad de generalización.
Por su parte, Elastic Net combina penalización L1 y L2, lo que permite reducir o incluso eliminar variables menos relevantes. Su efecto depende del valor del parámetro l1_ratio, que define el equilibrio entre ambas penalizaciones.

**Trade-off entre recall y precision desde el punto de vista del negocio**

Maximizar el recall implica reducir la cantidad de clientes que se van sin ser detectados por el modelo, lo cual es deseable desde el punto de vista del negocio. Sin embargo, esto puede aumentar los falsos positivos, es decir, clientes que el modelo identifica como posibles fugas pero que en realidad no se irían.
La curva Precision–Recall permite analizar este compromiso y ajustar el umbral de decisión de acuerdo con la capacidad operativa del equipo de retención y los costos asociados a contactar clientes innecesariamente.

**Otras técnicas que podrían mejorar el rendimiento**

Además de los enfoques evaluados, existen otras estrategias que podrían explorarse para mejorar el desempeño del modelo, como ajustar el umbral de decisión en lugar de utilizar el valor fijo de 0.5, incorporar pesos de clase para manejar el desbalance, aplicar técnicas de re-muestreo como sobremuestreo o submuestreo, o probar modelos más complejos y calibrar sus probabilidades.

## Conclusiones

Se desarrolló un flujo completo de preprocesamiento y modelamiento orientado a la predicción de churn mediante Regresión Logística. A lo largo del análisis, se comparó un modelo base con versiones que incorporan términos polinomiales y distintos esquemas de regularización, específicamente L2 y Elastic Net.

Dado el desbalance presente en la variable objetivo, se priorizó el uso de métricas como F1 y PR-AUC, las cuales entregan una evaluación más representativa del desempeño del modelo que la accuracy por sí sola. Finalmente, se observó que la regularización cumple un rol clave al controlar la complejidad introducida por las transformaciones polinomiales, contribuyendo además a una mayor estabilidad de los resultados a través de los distintos folds de la validación cruzada.